In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('C:\\Users\\joewa\\Work\\git\\vimms')
sys.path.append('C:\\Users\\Vinny\\work\\vimms')
sys.path.append('..')

In [ ]:
import numpy as np
import torch
import random as rand
import pylab as plt
import multiprocessing
from gym import spaces

from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.env_checker import check_env

In [ ]:
from vimms.Common import *
from vimms.Gym import VimmsGymEnv
from vimms.Evaluation import evaluate_simulated_env, evaluate_multiple_simulated_env
from vimms.ChemicalSamplers import MZMLFormulaSampler, MZMLRTandIntensitySampler
from vimms.Controller import SimpleTargetController

In [ ]:
np.random.seed(0)
rand.seed(0)
torch.manual_seed(0)

# 1. Parameters

In [ ]:
set_log_level_warning()

In [ ]:
# n_chemicals = (200, 500)
# mz_range = (100, 600)
# rt_range = (0, 300)
# intensity_range = (1E5, 1E10)

In [ ]:
# n_chemicals = (500, 2000)
# mz_range = (100, 600)
# rt_range = (200, 1000)
# intensity_range = (1E5, 1E10)

In [ ]:
n_chemicals = (200, 500)
mz_range = (100, 600)
rt_range = (200, 1000)
intensity_range = (1E5, 1E10)

In [ ]:
min_mz = mz_range[0]
max_mz = mz_range[1]
min_rt = rt_range[0]
max_rt = rt_range[1]
min_log_intensity = np.log(intensity_range[0])
max_log_intensity = np.log(intensity_range[1])

In [ ]:
isolation_window = 0.7
N = 10
rt_tol = 15
mz_tol = 10
min_ms1_intensity = 5000
ionisation_mode = POSITIVE
noise_density = 0.3
noise_max_val = 1e4

In [ ]:
in_dim = 50
out_dim = 50

# 2. Define Custom gym

In [ ]:
mzml_filename = 'Beer_multibeers_1_fullscan1.mzML'
mz_sampler = MZMLFormulaSampler(mzml_filename, min_mz=min_mz, max_mz=max_mz)
ri_sampler = MZMLRTandIntensitySampler(mzml_filename, min_rt=min_rt, max_rt=max_rt,
                                       min_log_intensity=min_log_intensity,
                                       max_log_intensity=max_log_intensity)

In [ ]:
params = {
    'chemical_creator': {
        'mz_range': mz_range,
        'rt_range': rt_range,
        'intensity_range': intensity_range,
        'n_chemicals': n_chemicals,
        'mz_sampler': mz_sampler,
        'ri_sampler': ri_sampler,
    },
    'noise': {
        'noise_density': noise_density,
        'noise_max_val': noise_max_val,
        'mz_range': mz_range
    },
    'env': {
        'ionisation_mode': ionisation_mode,
        'rt_range': rt_range,
        'N': N,
        'isolation_window': isolation_window,
        'mz_tol': mz_tol,
        'rt_tol': rt_tol,
        'min_ms1_intensity': min_ms1_intensity
    }
}

In [ ]:
class MaxIntensityEnv(VimmsGymEnv):
    def __init__(self, in_dim, out_dim, params):
        super().__init__(in_dim, out_dim, params)  
        self.last_excluded = []
        self.selected_precursors = []
    
    def _get_action_space(self):
        """
        Defines action space
        """
        return spaces.MultiBinary(self.out_dim)

    def _get_observation_space(self):
        """
        Defines observation space
        """
        intensity_high = np.log(intensity_range[1]) + 10
        features = {
            'intensity': spaces.Box(low=0.0, high=intensity_high, shape=(self.in_dim,)),
            'exclusion': spaces.Box(low=0.0, high=1.0, shape=(self.in_dim,))
        }
        return spaces.Dict(features)
    
    def _get_state(self, scan_to_process):
        """
        Converts a scan to a state
        """
        self.last_scan_to_process = scan_to_process        
        mzs, rt, intensities = self._get_mzs_rt_intensities(scan_to_process)
        
        precursors = []
        self.last_excluded = []
        for mz, intensity in zip(mzs, intensities):
            if self.controller.exclusion.is_excluded(mz, rt):
                excluded = 0
                self.last_excluded.append((mz, rt, intensity, ))                
            else:
                excluded = 1
            precursor = (mz, intensity, excluded, )
            precursors.append(precursor)                
        
        sorted_precursors = sorted(precursors, key=lambda item: item[1], reverse=True) # sort by intensity descending
        self.selected_precursors = []        
        feature_intensity = []
        feature_exclusion = []
        for i in range(self.in_dim): # get the first in_dim items
            precursor = sorted_precursors[i]
            mz, intensity, excluded = precursor            
            self.selected_precursors.append(precursor)
            
            intensity = np.log(intensity)
            if np.isnan(intensity):
                intensity = 0
            feature_intensity.append(intensity)
            feature_exclusion.append(excluded)
            
        feature_intensity = np.array(feature_intensity)
        feature_exclusion = np.array(feature_exclusion)
        features = {
            'intensity': feature_intensity,
            'exclusion': feature_exclusion
        }
        return features
    
    def _compute_reward(self, scan_to_process, results):
        """
        Computes fragmentation reward
        """
        parent_scan_id = self.controller.last_ms1_scan.scan_id
        assert scan_to_process.scan_id == parent_scan_id
    
        total_reward = 0.0
        for last_scan in results:
            if last_scan.ms_level >= 2:                
                precursor = last_scan.scan_params.get(ScanParameters.PRECURSOR_MZ)[0]
                mz = precursor.precursor_mz
                frag_rt = last_scan.scan_params.get(ScanParameters.METADATA)['frag_at']
                intensity = precursor.precursor_intensity
                
                # intensity reward
                min_intensity = self.controller.min_ms1_intensity
                reward = np.log(intensity) - np.log(min_ms1_intensity)
                
                # time reward
                time_filter = -1 if (mz, frag_rt, intensity, ) in self.last_excluded else 1
                reward = reward * time_filter
                
                total_reward += reward
        
        # use fewer fragmentations
        if len(results) > 0:
            total_reward /= len(results)
        return total_reward    

    def _take_action(self, action):
        """
        Modify controller variables based on the selected action
        """
        target_flag = action      
        assert len(target_flag) == len(self.selected_precursors)
        
        targets = []
        for i in range(len(target_flag)):
            t = target_flag[i]
            if t == 1:
                mz, intensity, excluded = self.selected_precursors[i]
                targets.append((mz, intensity))
                
        # self.seen_actions.update(['targets=%s' % targets])
        self.controller.targets = targets
        
    def _reset_controller(self, env_params):
        """
        Generates new controller
        """
        ionisation_mode = env_params['ionisation_mode']
        N = env_params['N']
        isolation_window = env_params['isolation_window']
        mz_tol = env_params['mz_tol']
        rt_tol = env_params['rt_tol']
        min_ms1_intensity = env_params['min_ms1_intensity']
        controller = SimpleTargetController(ionisation_mode, N, isolation_window, mz_tol, rt_tol, min_ms1_intensity)
        return controller

In [ ]:
set_log_level_info()

# 3. Training

### PPO

In [ ]:
env = MaxIntensityEnv(in_dim, out_dim, params)
check_env(env)

#### Multiprocessing

In [ ]:
from typing import Callable

def make_env(rank, seed=0):
    def _init():
        env = MaxIntensityEnv(in_dim, out_dim, params)
        env.seed(rank)
        return env
    set_random_seed(seed)
    return _init

num_cpu = multiprocessing.cpu_count()
num_cpu

In [ ]:
env = SubprocVecEnv([make_env(i) for i in range(num_cpu)]) 

#### Training

In [ ]:
model_name = 'PPO'
model = PPO("MultiInputPolicy", env, verbose=1, tensorboard_log='./results/%s_MaxIntensityEnv_tensorboard' % model_name)
model.learn(total_timesteps=250000)

In [ ]:
fname = 'results/%s_maxintensity_smallchems' % model_name
model.save(fname)

# 4. Evaluation

Load previously trained model

In [ ]:
model_name = 'PPO'
fname = 'results/%s_maxintensity_smallchems' % model_name
model = PPO.load(fname)

In [ ]:
num_episodes = 20
write_mzml_every = 5
N = 10

Generate some evaluation chemicals

In [ ]:
chems_list = []
for i in range(num_episodes):
    env = MaxIntensityEnv(in_dim, out_dim, params)
    env.reset()
    chems = env.chems
    print(len(chems))
    chems_list.append(chems)

Do a few topN steps manually

In [ ]:
def topN(observation, N=10, min_ms1_intensity=5000):
    intensities = observation['intensity']
    exclusions = observation['exclusion']
    log_min_ms1_intensity = np.log(min_ms1_intensity)
    
    action = []
    count = 0
    for i in range(len(intensities)):
        intensity, exclusion = intensities[i], exclusions[i]
        if count < N:
            if exclusion == 1 and intensity > log_min_ms1_intensity:
                action.append(1)
                count += 1
            else:
                action.append(0)
        else:
            action.append(0)
    action = np.array(action)
    return action

In [ ]:
env = MaxIntensityEnv(in_dim, out_dim, params)
chems = chems_list[0]
observation = env.reset(chems=chems)
observation

In [ ]:
for i in range(10):
    action = topN(observation, N=N, min_ms1_intensity=min_ms1_intensity)
    observation, reward, done, info = env.step(action)
    action, reward, observation, done
    print('step %d' % i)
    print('action = %s' % action)
    print('reward = %f' % reward)
    print('done = %s' % done)    
    print('next observation = %s' % observation)
    print()

Compute evaluation here

In [ ]:
def evaluation(model, model_name, in_dim, out_dim, params, num_episodes, chems_list):
    env = MaxIntensityEnv(in_dim, out_dim, params)
    total_rewards = []
    total_reward_per_chems = []
    env_list = []
    
    for i_episode in range(num_episodes):
        chems = chems_list[i_episode]
        observation = env.reset(chems=chems)
        total_reward = 0
        for t in range(1000):
            # env.render()
            if model_name == 'random':
                action = env.action_space.sample()
                out_file = 'test_%s_%d.mzML' % ('random', i_episode)
            elif model_name == 'TopN':
                action = topN(observation, N=N, min_ms1_intensity=min_ms1_intensity)
                out_file = 'test_%s_%d.mzML' % ('TopN', i_episode)                
            else:
                action, _ = model.predict(observation, deterministic=True)     
                out_file = 'test_%s_%d.mzML' % (model_name, i_episode)
                                                
            observation, reward, done, info = env.step(action)
            total_reward += reward
            if done:
                seen_actions = env.seen_actions.most_common()
                n_chems = len(env.chems)
                reward_per_chems = total_reward / n_chems
                total_rewards.append(total_reward)
                total_reward_per_chems.append(reward_per_chems)
                env_list.append(env.vimms_env)
                if i_episode % write_mzml_every == 0 or i_episode == num_episodes-1:
                    env.vimms_env.write_mzML('results', out_file)
                print('Episode %d timesteps %d reward %f n_chems %d reward/chems %f' % (i_episode, t+1, total_reward, n_chems, reward_per_chems))                                        
                break
    env.close()
    print('Average total reward = %f' % np.mean(total_rewards))
    return np.array(total_rewards), np.array(total_reward_per_chems), env_list

In [ ]:
model_total_rewards, model_reward_per_chems, model_env_list = evaluation(model, model_name, in_dim, out_dim, params, num_episodes, chems_list)

In [ ]:
topN_total_rewards, topN_reward_per_chems, topN_env_list = evaluation(None, 'TopN', in_dim, out_dim, params, num_episodes, chems_list)

In [ ]:
random_total_rewards, random_reward_per_chems, random_env_list = evaluation(None, 'random', in_dim, out_dim, params, num_episodes, chems_list)

### Plots

In [ ]:
def plot_diff(controller_names, scores_list, ref_name, ref_scores):
    for controller_name, scores in zip(controller_names, scores_list):
        diff = scores - ref_scores
        perc = np.multiply(diff, 1/ref_scores) * 100
        plt.plot(diff, label=controller_name)
    plt.title('Score improvement over %s' % ref_name)
    plt.ylabel('Score Improvement (%)')
    plt.xlabel('Episode')        
    plt.legend()

def plot_arr(controller_names, arr_list, title):
    for controller_name, arr in zip(controller_names, arr_list):
        plt.plot(arr, label=controller_name)
    plt.title('%s per Episode' % title)
    plt.ylabel(title)
    plt.xlabel('Episode')        
    plt.legend()
        
def get_scores(env_list, type='both'):
    scores = []
    for env in env_list:
        score = get_score(env, type=type)
        scores.append(score)
    return np.array(scores)

def get_score(env, type='both'):
    res = evaluate_simulated_env(env)
    if type == 'both':
        score = res['coverage_proportion'] * res['intensity_proportion']
    elif type == 'coverage':
        score = res['coverage_proportion']    
    elif type == 'intensity':
        score = res['intensity_proportion']    
    return score

In [ ]:
plot_arr([model_name, 'TopN', 'Random'], [model_total_rewards, topN_total_rewards, random_total_rewards], 'Total Rewards')

In [ ]:
plot_arr([model_name, 'TopN', 'Random'], [model_reward_per_chems, topN_reward_per_chems, random_reward_per_chems], 'Reward/Chems')

In [ ]:
score_type = 'coverage'
model_scores = get_scores(model_env_list, type=score_type)
topN_scores = get_scores(topN_env_list, type=score_type)
random_scores = get_scores(random_env_list, type=score_type)
plot_arr([model_name, 'TopN', 'Random'], [model_scores, topN_scores, random_scores], 'Scores (coverage)')

In [ ]:
score_type = 'intensity'
model_scores = get_scores(model_env_list, type=score_type)
topN_scores = get_scores(topN_env_list, type=score_type)
random_scores = get_scores(random_env_list, type=score_type)
plot_arr([model_name, 'TopN', 'Random'], [model_scores, topN_scores, random_scores], 'Scores (intensity)')